In [1]:
import os

In [2]:
dataset_dir = '../../dataset_resized/test/test/'
# dataset_dir = '../../dataset_original/test/test/'

In [3]:
imgs = os.listdir(dataset_dir)

In [4]:
import numpy as np
import pandas as pd

In [5]:
from skimage.io import imread, imshow
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

In [6]:
from tqdm import tqdm

In [7]:
import h5py

In [8]:
X = []
for i in tqdm(range(len(imgs))):
    img = imgs[i]
    arr = imread(dataset_dir+img, as_grey=True)
    # arr = cv2.resize(arr, (80, 60))
    arr = list(arr.flatten())
    X.append(arr)

100%|██████████| 79726/79726 [04:07<00:00, 321.84it/s]


In [9]:
X = np.array(X)

In [10]:
with h5py.File("X.h5") as h:
    h.create_dataset("X", data=X)

In [11]:
X.shape

(79726, 3072)

In [12]:
from sklearn.neighbors import NearestNeighbors

In [13]:
nbrs = NearestNeighbors(n_neighbors=10, algorithm='ball_tree').fit(X)

In [14]:
distances, indices = nbrs.kneighbors(X)

In [16]:
for i in tqdm(range(len(imgs))):
    imgs[i] = imgs[i].encode()    

100%|██████████| 79726/79726 [00:00<00:00, 693341.38it/s]


In [17]:
imgs[0]

b'img_1.jpg'

In [18]:
with h5py.File("neighbors.h5") as h:
    h.create_dataset("list", data=imgs)
    h.create_dataset('distance', data=distances)
    h.create_dataset('indices', data=indices)

In [19]:
distances = np.array(distances)

In [20]:
distances.shape

(79726, 10)

In [24]:
distances[0]

array([ 0.        ,  8.56158347,  8.72771176,  8.8001842 ,  8.83867031,
        8.94023227,  9.02856122,  9.0524942 ,  9.07960821,  9.08424594])

In [21]:
indices = np.array(indices)

In [22]:
indices.shape

(79726, 10)

In [26]:
indices[0]

array([    0, 76405, 42049, 46259, 57421, 33548, 68129, 74261, 48321, 77195], dtype=int64)

checkpoint

In [ ]:
import h5py

In [ ]:
import pandas as pd
from tqdm import tqdm

In [45]:
with h5py.File("neighbors.h5", 'r') as h:
    d = np.array(h['distance'])
    idx = np.array(h['indices'])
    im = np.array(h['list'])

In [55]:
ims = []
for i in tqdm(range(len(im))):
    ims.append(im[i].decode())
ims = np.array(ims)

100%|██████████| 79726/79726 [00:00<00:00, 641156.32it/s]


In [48]:
filename = '../submissions/submission_vgg16_resnet50_avg.csv'

In [ ]:
df = pd.read_csv(filename)

In [84]:
cls = list(df)[1:]

In [148]:
(df.img == ims).mean()

1.0

In [194]:
def knn_out(filename, K, weight, filename_new):
    df = pd.read_csv(filename)
    # print(df.head())
    pdt = np.array(df[cls])
    pdt0 = np.array(df[cls])
    # print(pdt[1])
    for k in range(1,K):
        # print(weight[k])
        # print(pdt[idx[:,k]][1])
        pdt += pdt0[idx[:,k]] * weight[k]
        # print(pdt[1])
        # print(pdt0[1])
    # print(np.sum(weight[:K]))
    pdt /= np.sum(weight[:K])
    # print(pdt[1])
    df[cls] = pdt
    # print(df.head())
    df.to_csv(filename_new, index=False)

In [166]:
filename_new = '../submissions/submission_vgg16_resnet50_avg_2nn.csv'

In [175]:
weight = np.array([1-x*0.1 for x in range(10)])

In [195]:
for K in tqdm(range(2, 11)):
    filename_new_1 = '../submissions/submission_vgg16_resnet50_avg_'+str(K)+'nn.csv'
    knn_out(filename, K, np.ones(10), filename_new_1)
    filename_new_2 = '../submissions/submission_vgg16_resnet50_avg_'+str(K)+'nn_w.csv'
    knn_out(filename, K, weight, filename_new_2)

100%|██████████| 9/9 [01:01<00:00,  7.37s/it]
